In [1]:
!pip install delta-spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [2]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

In [3]:
spark_depenedencies_jars = [
    "org.apache.hadoop:hadoop-common:3.3.1",
    "org.apache.hadoop:hadoop-client:3.3.1",
    "org.apache.hadoop:hadoop-aws:3.3.1",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1",
    "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.1",
    "io.delta:delta-core_2.12:2.2.0"
]
spark_depenedencies_jars_str = ",".join(spark_depenedencies_jars)

In [4]:
SPARK_MASTER = "spark://spark-master:7077"
SPARK_MASTER_LOCAL = "local"
HIVE_METASTORE_URI_LOCAL = "thrift://localhost:9083"
HIVE_METASTORE_URI = "thrift://hive-metastore:9083"

In [5]:
aws_access_key = "minio"
aws_secret_key = "minio123"
aws_endpoint = "http://minio:9000"

In [6]:
spark = SparkSession.builder \
        .appName("Delta Upsert POC") \
        .master(SPARK_MASTER_LOCAL) \
        .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
        .config("spark.hadoop.fs.s3a.endpoint", aws_endpoint) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.databricks.delta.retentionDurationCheck.enabled", "false") \
        .config("spark.hadoop.hive.metastore.uris", HIVE_METASTORE_URI) \
        .config("spark.sql.catalogImplementation", "hive") \
        .config("spark.jars.packages", spark_depenedencies_jars_str) \
        .enableHiveSupport() \
        .getOrCreate()

In [7]:
BASE_DELTA_DIR_S3 = "s3a://warehouse/delta/inventory/"

In [8]:
customers_delta_dir = BASE_DELTA_DIR_S3 + "customers"
orders_delta_dir = BASE_DELTA_DIR_S3 + "orders"
products_delta_dir = BASE_DELTA_DIR_S3 + "products"

In [9]:
old_customers_table = DeltaTable.forPath(spark, customers_delta_dir).toDF()
old_orders_table = DeltaTable.forPath(spark, orders_delta_dir).toDF()
old_products_table = DeltaTable.forPath(spark, products_delta_dir).toDF()

new_customers_table = DeltaTable.forPath(spark, customers_delta_dir)
new_orders_table = DeltaTable.forPath(spark, orders_delta_dir)
new_products_table = DeltaTable.forPath(spark, products_delta_dir)

In [12]:
new_customers_table.alias('new_customers').merge(old_customers_table.alias('old_customers'), "old_customers.id = new_customers.id and new_customers.curr_timestamp < old_customers.curr_timestamp").whenMatchedDelete().execute()
new_orders_table.alias('new_orders').merge(old_orders_table.alias('old_orders'), "old_orders.order_number = new_orders.order_number and new_orders.curr_timestamp < old_orders.curr_timestamp").whenMatchedDelete().execute()
new_products_table.alias('new_products').merge(old_products_table.alias('old_products'), "old_products.id = new_products.id and new_products.curr_timestamp < old_products.curr_timestamp").whenMatchedDelete().execute()

In [13]:
customer_csv_location = "s3a://data/new_customers.csv"

In [14]:
new_customers_csv = spark.read.format("com.databricks.spark.csv") \
                               .option("header", "true") \
                               .option("inferSchema", "true") \
                               .option("delimiter", ",") \
                               .option("mode", "DROPMALFORMED") \
                               .load(customer_csv_location)

In [16]:
new_customers_csv.show(truncate=False)

+----+----------+---------+-------------------------+---+-----------------------+
|id  |first_name|last_name|email                    |op |curr_timestamp         |
+----+----------+---------+-------------------------+---+-----------------------+
|1006|Eugene    |Goldberg |eugene.goldberg@gmail.com|c  |2023-02-12 07:12:27.566|
|1005|Rahul     |Sahoo    |rahulsahoo1999@gmail.com |u  |2025-02-12 07:13:27.566|
+----+----------+---------+-------------------------+---+-----------------------+



In [17]:
new_customers_table.alias('customers').merge(new_customers_csv.alias('customers_csv'), "customers_csv.id = customers.id and customers.curr_timestamp < customers_csv.curr_timestamp").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()